In [8]:

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import itertools
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
from pandas.io.json import json_normalize


### 使用自定义的yield方式读取

In [26]:
global_path = "/home/zhoutong/data/kaggle/train.csv"
def _gen():
    with open(global_path,"r+") as f:
        for line in f:
            yield line.strip("\n")

def get_generator():
    return _gen()
gen = get_generator()
tmp = list(itertools.islice(gen,0,10))
list(map(lambda x:len(x.split(",")), tmp))

[12, 45, 45, 45, 45, 45, 45, 45, 45, 45]

### 使用pandas的API进行yield读取

In [124]:
json_columns=['device', 'geoNetwork', 'totals', 'trafficSource']
converter_dict = {col : json.loads for col in json_columns}
converter_dict
reader = pd.read_csv(global_path, delimiter=",", converters=converter_dict, chunksize=300, dtype={"fullVisitorId":"str"})
df = reader.get_chunk()

{'device': <function json.loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>,
 'geoNetwork': <function json.loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>,
 'totals': <function json.loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>,
 'trafficSource': <function json.loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>}

In [125]:
for col in json_columns:
    col_as_df = json_normalize(df[col])
    # f-string 3.6的新功能
    col_as_df.columns = [f"{col}.{subcol}"  for subcol in col_as_df.columns]
    df = df.drop(col, axis=1).merge(col_as_df, right_index=True, left_index=True)
    
df.columns
df.head(3)


Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews', 'totals.visits',
       'trafficS

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [121]:
df1 = pd.DataFrame(data=[[1,2,3],[2,3,4]], columns=["a","b","c"])
df2 = pd.DataFrame(data=[[33,22,66],[86,57,4]], columns=["a1","b1","c1"])
df1
df2
df1.merge(df2, left_index=True,right_index=True)


,a,b,c
0,1,2,3
1,2,3,4


,a1,b1,c1
0,33,22,66
1,86,57,4


,a,b,c,a1,b1,c1
0,1,2,3,33,22,66
1,2,3,4,86,57,4
